<a href="https://colab.research.google.com/github/danieltavares24/DETECCAO_DE_FRAUDES_COM_CARTOES_DE_CREDITO/blob/main/modelo_randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Desafio Técnico – Hackathon Forecast Big Data 2025**

# **Bibliotecas**

In [3]:
# Bibliotecas necessárias
!pip install pandas numpy scikit-learn xgboost lightgbm seaborn matplotlib


# **Arquivos**

In [4]:
import pandas as pd

# Arquivos Parquet
transacoes = pd.read_parquet('transacoes_2022.parquet')
produtos = pd.read_parquet('cadastro_produtos.parquet')
pdvs = pd.read_parquet('cadastro_pdvs.parquet')

# **Insights**

In [5]:
# Data para datetime
transacoes['transaction_date'] = pd.to_datetime(transacoes['transaction_date'])

# Coluna de semana
transacoes['Semana'] = transacoes['transaction_date'].dt.isocalendar().week

# Vendas por semana, pdv e produto
vendas_semanais = transacoes.groupby(['Semana', 'internal_store_id', 'internal_product_id'])['quantity'].sum().reset_index()

# **Esquema**

In [6]:
print("Colunas de vendas_semanais:", vendas_semanais.columns.tolist())
print("Colunas de produtos:", produtos.columns.tolist())
print("Colunas de pdvs:", pdvs.columns.tolist())

Colunas de vendas_semanais: ['Semana', 'internal_store_id', 'internal_product_id', 'quantity']
Colunas de produtos: ['produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante']
Colunas de pdvs: ['pdv', 'premise', 'categoria_pdv', 'zipcode']


# **Padronização**

In [7]:
# Colunas em vendas_semanais para bater com os cadastros
vendas_semanais.rename(columns={
    'internal_store_id': 'pdv',
    'internal_product_id': 'produto'
}, inplace=True)

# **Enriquecimento**

In [8]:
# Cadastro de produtos
vendas_semanais = vendas_semanais.merge(produtos, on='produto', how='left')

# Cadastro de PDVs
vendas_semanais = vendas_semanais.merge(pdvs, on='pdv', how='left')

# **Codificação**

In [9]:
from sklearn.preprocessing import LabelEncoder

# Colunas com poucas categorias
label_cols = ['categoria', 'subcategoria', 'marca', 'fabricante', 'categoria_pdv', 'premise']

for col in label_cols:
    if col in vendas_semanais.columns:
        le = LabelEncoder()
        vendas_semanais[col] = le.fit_transform(vendas_semanais[col].astype(str))

# **Variáveis de histórico**

In [10]:
# Consistência temporal
vendas_semanais = vendas_semanais.sort_values(by=['pdv', 'produto', 'Semana'])

# Média móvel das últimas 4 semanas
vendas_semanais['media_4'] = (
    vendas_semanais
    .groupby(['pdv', 'produto'])['quantity']
    .transform(lambda x: x.shift(1).rolling(4).mean())
)

# Tendência (diferença entre semanas)
vendas_semanais['tendencia'] = (
    vendas_semanais
    .groupby(['pdv', 'produto'])['quantity']
    .transform(lambda x: x.diff())
)

# NaNs
vendas_semanais.fillna(0, inplace=True)

# **Treino e base de previsão**

In [11]:
# Treino: semanas 1 a 52
treino = vendas_semanais[vendas_semanais['Semana'] <= 52]

# Base de previsão: todos os pares pdv/produto para semanas 1 a 5
base_pred = (
    treino[['pdv', 'produto']]
    .drop_duplicates()
    .assign(Semana=1)
)

base_pred = pd.concat([
    base_pred.assign(Semana=s) for s in range(1, 6)
], ignore_index=True)

# Atributos
base_pred = base_pred.merge(produtos, on='produto', how='left')
base_pred = base_pred.merge(pdvs, on='pdv', how='left')

# Aplicar mesma codificação
for col in label_cols:
    if col in base_pred.columns:
        le = LabelEncoder()
        base_pred[col] = le.fit_transform(base_pred[col].astype(str))

# **Treinamento de outros modelos**

In [12]:
# Colunas para treino
features = ['media_4', 'tendencia', 'categoria', 'subcategoria', 'marca', 'fabricante', 'categoria_pdv', 'premise']
X = vendas_semanais[features]
y = vendas_semanais['quantity']

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.4 MB/s eta 0:00:00


In [15]:
from catboost import CatBoostRegressor

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

modelos = {
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Ridge': Ridge(),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(verbose=0, random_state=42)
}

resultados = {}

for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    pred = modelo.predict(X_val)
    mae = mean_absolute_error(y_val, pred)
    resultados[nome] = mae

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.697383 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 996
[LightGBM] [Info] Number of data points in the train set: 4997525, number of used features: 8
[LightGBM] [Info] Start training from score 8.529178


# **Comparação dos resultados**

In [17]:
for nome, erro in sorted(resultados.items(), key=lambda x: x[1]):
    print(f'{nome}: MAE = {erro:.2f}')

RandomForest: MAE = 6.49
LightGBM: MAE = 7.05
CatBoost: MAE = 7.10
XGBoost: MAE = 7.13
Ridge: MAE = 12.13


# **Gerar media_4 e tendencia para base_pred**

In [19]:
# Histórico com base_pred
base_pred = base_pred.merge(
    vendas_semanais[['pdv', 'produto', 'Semana', 'media_4', 'tendencia']],
    on=['pdv', 'produto', 'Semana'],
    how='left'
)

# NaNs (caso não haja histórico suficiente)
base_pred[['media_4', 'tendencia']] = base_pred[['media_4', 'tendencia']].fillna(0)

# **Confirmar que as features estão completas**

In [20]:
print(base_pred[features].isnull().sum())

media_4          0
tendencia        0
categoria        0
subcategoria     0
marca            0
fabricante       0
categoria_pdv    0
premise          0
dtype: int64


In [21]:
base_pred['quantidade'] = modelos['RandomForest'].predict(base_pred[features]).round().astype(int)